In [ ]:
# Set up colab instance
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install pydicom

     |████████████████████████████████| 1.9MB 13.1MB/s 


In [ ]:
import os
os.chdir('drive/MyDrive/GIF-7005-Projet/gif-7005-projet/')

In [ ]:
import time

import torch

from dataset import *
from models import *
from metrics import *

torch.manual_seed(111)

In [ ]:
N_EPOCH = 10
BATCH_SIZE = 32

In [ ]:
dataset = dcm_dataset('data')

train_set, test_set = train_test_split(dataset)
test_set.transforms = Downsample()

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=BATCH_SIZE, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=1, num_workers=2)

In [ ]:
class VGG16(nn.Module):
    def __init__(self):

        super().__init__()
        pretrained = True
        # resample to 224 x 224 spatial sizes
        self.upsample = nn.Upsample(
            size=(224, 224), mode='bilinear', align_corners=True)

        self.base_model = torch.hub.load(
            'pytorch/vision:v0.6.0', 'vgg16', pretrained=True)

        # reshape the ouput layer
        self.base_model.classifier[-1] = nn.Linear(in_features=4096, out_features=1)
        self.base_model.classifier.requires_grad = True

        # freeze everything but the last layer
        for name, param in self.named_parameters():
            if "classifier" in name:
                param.requires_grad = True
            else:
                param.requires_grad = not pretrained

    def forward(self, x):
        out = self.upsample(x)

        # To simulate 3 channels
        out = torch.cat([out, out, out], dim=1)
        out = self.base_model(out)

        return out


In [ ]:
from torchsummary import summary

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cnn = VGG16().to("cuda:0")
optim = torch.optim.Adam(cnn.parameters(), lr=1e-4)
loss_fn = torch.nn.BCEWithLogitsLoss()

summary(cnn, (1, 224, 224))


Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
          Upsample-1          [-1, 1, 224, 224]               0
            Conv2d-2         [-1, 64, 224, 224]           1,792
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
              ReLU-5         [-1, 64, 224, 224]               0
         MaxPool2d-6         [-1, 64, 112, 112]               0
            Conv2d-7        [-1, 128, 112, 112]          73,856
              ReLU-8        [-1, 128, 112, 112]               0
            Conv2d-9        [-1, 128, 112, 112]         147,584
             ReLU-10        [-1, 128, 112, 112]               0
        MaxPool2d-11          [-1, 128, 56, 56]               0
           Conv2d-12          [-1, 256, 56, 56]         295,168
             ReLU-13          [-1, 256, 56, 56]               0
           Conv2d-14          [-1, 256

In [ ]:
# If model exists, load it
if os.path.exists("vgg16.pt"):
  print("loaded")
  cnn.load_state_dict(torch.load("vgg16.pt"))

loaded


In [ ]:
# params = list(cnn.parameters()) + list(regressor.parameters())
optim = torch.optim.Adam(cnn.parameters(), lr=1e-4)

loss_fn = torch.nn.BCEWithLogitsLoss()

In [ ]:
cnn.train()

for epoch in range(N_EPOCH):
    timer = time.time()

    img_processed = 0

    for img, (target, bounding_box) in train_loader:
        optim.zero_grad()
        img, target = img.to("cuda:0"), target.to("cuda:0")

        pred  = cnn(img)

        loss = loss_fn(pred.squeeze(), target)
        loss.backward()

        optim.step()

        img_processed += img.shape[0]

    print("Epoch : {}".format(epoch + 1))
    print("Time elapsed : {}".format(time.time() - timer))

Epoch : 1
Time elapsed : 2557.5021402835846
Epoch : 2
Time elapsed : 427.76289105415344
Epoch : 3
Time elapsed : 428.2250370979309
Epoch : 4
Time elapsed : 430.9574086666107
Epoch : 5
Time elapsed : 429.93483805656433
Epoch : 6
Time elapsed : 432.68209314346313
Epoch : 7
Time elapsed : 433.13053846359253
Epoch : 8
Time elapsed : 430.760865688324
Epoch : 9
Time elapsed : 430.8357951641083
Epoch : 10
Time elapsed : 432.5048496723175
Epoch : 11
Time elapsed : 437.2467505931854
Epoch : 12
Time elapsed : 443.94202065467834
Epoch : 13
Time elapsed : 446.03476190567017
Epoch : 14
Time elapsed : 444.90193462371826
Epoch : 15
Time elapsed : 450.2874004840851
Epoch : 16
Time elapsed : 448.0620198249817
Epoch : 17
Time elapsed : 430.97594928741455
Epoch : 18
Time elapsed : 433.25922107696533
Epoch : 19
Time elapsed : 447.10144448280334
Epoch : 20
Time elapsed : 445.5016288757324
Epoch : 21
Time elapsed : 445.45686745643616
Epoch : 22
Time elapsed : 445.490975856781
Epoch : 23
Time elapsed : 446.0

In [ ]:
# Accuracy

cnn.eval()
# regressor.eval()

VGG16(
  (upsample): Upsample(size=(224, 224), mode=bilinear)
  (base_model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), p

In [ ]:
# Save models
torch.save(cnn.state_dict(), "vgg16.pt")

In [ ]:
cnn.eval()
run_test(cnn, test_loader, [Confusion_matrix()], "cuda:0")

tensor([[3783.,  726.],
        [ 376.,  451.]])
